#### Najpre ucitajmo dataset sa mog githaba (ne mora uvek da se ucitava lokalno sa uredjaja, moze i sa interneta)

In [13]:
import pandas as pd
brit = pd.read_csv('https://raw.githubusercontent.com/Niecmano/sempandas/main/data/brit_sredjen.csv')
print(brit.shape)
brit.sample(13)

(1076, 27)


,Title,AlbumName,Record Label,Performer,Track.number,Song.duration,Composer,Year,Genre,Styles,...,tempo,valence,duration_ms,popularity,shake.the.audience,romantic,obscene,family.spiritual,sadness,First composer
174,You Like Me Too Much,Help!,Apple Records / Capitol,The Beatles,10,2:36,George Harrison,1965,"Pop/Rock, Stage & Screen","British Invasion, Contemporary Pop/Rock, Rock ...",...,88.316,0.899,156867.0,48.0,0.000658,0.131299,0.173864,0.000658,0.156742,George Harrison
638,You're So Good for Me,A-Tom-Ic Jones,Decca / Universal,Tom Jones,12,2:21,Ezio Leoni,1966,"Pop/Rock, Vocal","AM Pop, Contemporary Pop/Rock, Blue-Eyed Soul,...",...,113.655,0.630,140960.0,8.0,0.001834,0.095359,0.130542,0.000907,0.000907,Ezio Leoni
862,Each Time,Take Me for What I'm Worth,Pye,The Searchers,9,2:48,Jackie DeShannon,1965,Pop/Rock,"British Invasion, Contemporary Pop/Rock, Folk-...",...,128.790,0.557,167640.0,3.0,0.001548,0.000892,0.002024,0.000774,0.001120,Jackie DeShannon
1013,I'm a Man,Five Live Yardbirds,Sunspots,The Yardbirds,9,4:33,Bo Diddley,1964,"Pop/Rock, Blues","Blues-Rock, British Blues, British Invasion, R...",...,203.683,0.935,201307.0,0.0,0.000572,0.002632,0.005848,0.026181,0.081081,Bo Diddley
15,Blue Monday,American Tour,BMG,The Dave Clark Five,6,2:56,Dave Clark / Mike Smith,1964,Pop/Rock,"British Invasion, Early Pop/Rock, Rock & Roll",...,160.349,0.642,136827.0,2.0,0.000797,0.000797,0.000797,0.074552,0.092456,Dave Clark
848,Can't Help Forgiving You,It's the Searchers,BMG / Castle Communications / Sony BMG,The Searchers,12,2:05,Jackie DeShannon,1964,Pop/Rock,"British Invasion, Contemporary Pop/Rock, Merse...",...,106.396,0.855,125467.0,1.0,0.000849,0.000849,0.000849,0.000849,0.000849,Jackie DeShannon
865,Take Me for What I'm Worth,Take Me for What I'm Worth,Pye,The Searchers,12,2:42,P.F. Sloan,1965,Pop/Rock,"British Invasion, Contemporary Pop/Rock, Folk-...",...,140.404,0.710,160440.0,29.0,0.001595,0.001462,0.001144,0.000741,0.002024,P.F. Sloan
84,Blueberry Hill,You Got What It Takes,BMG,The Dave Clark Five,9,2:16,Al Lewis / Vincent Rose / Larry Stock,1967,Pop/Rock,"British Invasion, Rock & Roll",...,109.946,0.447,136667.0,2.0,0.001224,0.032874,0.001224,0.033149,0.045565,Al Lewis
380,The Monkey Time,Fame at Last,Polydor / Universal,Georgie Fame,3,2:41,Curtis Mayfield,1965,"Pop/Rock, R&B, Jazz, Blues","British Invasion, Blue-Eyed Soul, Soul, Vocal ...",...,133.159,0.727,161400.0,4.0,0.001074,0.001074,0.001074,0.060264,0.188855,Curtis Mayfield
872,Don't Be That Way,Sandie,BMG / Union Square Music,Sandie Shaw,6,2:08,Chris Andrews,1965,Pop/Rock,"British Invasion, Girl Groups, AM Pop",...,135.633,0.661,128400.0,4.0,0.001053,0.071985,0.001284,0.024663,0.001253,Chris Andrews


#### Dati pandas df cemo prebaciti u spark.. Pravimo semu datafrejma - StructType koji sadrzi StructField za svaku kolonu.

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, DateType

spark = SparkSession.builder.appName("Britanski bendovi").getOrCreate()
brit.rename(columns={'Song.duration':'duration'}, inplace=True)
schema = StructType([StructField(dataType=StringType(),name='Title',nullable=True),
                StructField(dataType=StringType(),name='Performer',nullable=True),
                StructField(dataType=DateType(),name='duration',nullable=True),
                StructField(dataType=StringType(),name='AlbumName',nullable=True),StructField(dataType=StringType(),name='Genre',nullable=True),StructField(dataType=FloatType(),name='energy',nullable=True)])
brit['Title']=brit['Title'].astype(str)
brit['AlbumName']=brit['AlbumName'].astype(str)
brit['Performer']=brit['Performer'].astype(str)
# brit.dtypes
bsp = spark.createDataFrame(brit[schema.fieldNames()])
print(type(bsp))
bsp.show(300)

NameError: name 'brit' is not defined

In [35]:
bsp.select(['Title','duration']).show(10)

+--------------------+--------+
|               Title|duration|
+--------------------+--------+
|     Glad All Over  |    2:43|
|   All of the Time  |    2:15|
|   Crying Over You  |    2:09|
|              Stay  |    2:09|
|          Chaquita  |    2:13|
|     Do You Love Me |    2:43|
|   Bits and Pieces  |    1:58|
|        I Know You  |    2:00|
|              Time  |    2:19|
|Who Do You Think ...|    2:10|
+--------------------+--------+
only showing top 10 rows



In [26]:
bsp.groupby('AlbumName').agg({"energy": "SUM"}).show()

+--------------------+------------------+
|           AlbumName|       sum(energy)|
+--------------------+------------------+
|          5 X 5 [US]|5.4830000000000005|
|   Weekend in London| 7.395999999999998|
|  Satisfied with You|             6.331|
|       American Tour|             6.327|
|        Try Too Hard|             7.408|
|       Glad All Over|7.4190000000000005|
| I Like It Like That|             2.086|
|      Coast to Coast|             7.177|
|Magical Mystery Tour| 4.609999999999999|
|You Got What It T...|             6.696|
|    The Animals [US]| 7.351999999999999|
|           Animalism|             0.564|
|     Winds of Change| 5.646999999999999|
| The Animals on Tour|             7.727|
|       Animalization|             8.324|
|            Revolver|             7.428|
|  A Hard Day's Night|             7.924|
|               Help!|              8.77|
|    Beatles for Sale| 6.987000000000001|
|         Rubber Soul| 7.550999999999999|
+--------------------+------------

In [36]:
from pyspark.sql.functions import udf
def usek(traj):
    min,sek = map(int,traj.split(':'))
    return min*60+sek
usec = udf(usek, IntegerType())

novi = bsp.withColumn('trajanje',usec(bsp['duration']))
novi.show()

+--------------------+-------------------+--------+-------------+--------+------+--------+
|               Title|          Performer|duration|    AlbumName|   Genre|energy|trajanje|
+--------------------+-------------------+--------+-------------+--------+------+--------+
|     Glad All Over  |The Dave Clark Five|    2:43|Glad All Over|Pop/Rock| 0.803|     163|
|   All of the Time  |The Dave Clark Five|    2:15|Glad All Over|Pop/Rock| 0.667|     135|
|   Crying Over You  |The Dave Clark Five|    2:09|Glad All Over|Pop/Rock|  0.36|     129|
|              Stay  |The Dave Clark Five|    2:09|Glad All Over|Pop/Rock| 0.926|     129|
|          Chaquita  |The Dave Clark Five|    2:13|Glad All Over|Pop/Rock| 0.891|     133|
|     Do You Love Me |The Dave Clark Five|    2:43|Glad All Over|Pop/Rock| 0.913|     163|
|   Bits and Pieces  |The Dave Clark Five|    1:58|Glad All Over|Pop/Rock| 0.835|     118|
|        I Know You  |The Dave Clark Five|    2:00|Glad All Over|Pop/Rock| 0.872|     120|

In [40]:
spark.sql("DROP VIEW IF EXISTS brits")
novi.createTempView('brits')
spark.sql('SELECT * from brits where energy<0.6 and trajanje>150').show()

+--------------------+--------------------+--------+-------------------+---------------+------+--------+
|               Title|           Performer|duration|          AlbumName|          Genre|energy|trajanje|
+--------------------+--------------------+--------+-------------------+---------------+------+--------+
|Whenever You're A...| The Dave Clark Five|    2:54|      American Tour|       Pop/Rock| 0.493|     174|
|       Blue Monday  | The Dave Clark Five|    2:56|      American Tour|       Pop/Rock|  0.52|     176|
|         Come Home  | The Dave Clark Five|    2:48|  Weekend in London|       Pop/Rock| 0.361|     168|
|    Hurting Inside  | The Dave Clark Five|    2:37|  Weekend in London|       Pop/Rock| 0.354|     157|
|I'll Be Yours My ...| The Dave Clark Five|    2:42|I Like It Like That|       Pop/Rock| 0.524|     162|
| Sitting Here Baby  | The Dave Clark Five|    3:00|         5 X 5 [US]|       Pop/Rock|  0.43|     180|
|You Don't Want My...| The Dave Clark Five|    2:48|   

In [2]:
spark.stop()